In [1]:
import pandas as pd

In [4]:
parse_dates = [
    "lpep_pickup_datetime",
    "lpep_dropoff_datetime"
]

In [35]:
df1 = pd.read_parquet("green_tripdata_2025-11.parquet", 
                      # parse_dates=parse_dates
                     )
df1.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [3]:
df1.shape

(46912, 21)

In [9]:
df1.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [18]:
df1_ftr = df1[(df1['lpep_pickup_datetime'] >= '2025-11-01') & (df1['lpep_pickup_datetime'] <= '2025-12-01') &
    (df1['trip_distance'] <= 1)]
df1_ftr.shape

(8007, 21)

In [19]:
df1['trip_distance'].max()

np.float64(172273.8)

In [20]:
df_tripless_100 = df1[df1['trip_distance'] < 100]

In [28]:
df_tripless_100.loc[df_tripless_100['trip_distance'].idxmax()]

VendorID                                   2
lpep_pickup_datetime     2025-11-14 15:36:27
lpep_dropoff_datetime    2025-11-14 18:40:48
store_and_fwd_flag                         N
RatecodeID                               4.0
PULocationID                             130
DOLocationID                             265
passenger_count                          2.0
trip_distance                          88.03
fare_amount                            610.6
extra                                    0.0
mta_tax                                  0.5
tip_amount                               0.0
tolls_amount                             0.0
ehail_fee                                NaN
improvement_surcharge                    1.0
total_amount                           612.1
payment_type                             2.0
trip_type                                1.0
congestion_surcharge                     0.0
cbd_congestion_fee                       0.0
Name: 18867, dtype: object

In [30]:
##Read lookup location
df_location = pd.read_csv("taxi_zone_lookup.csv")
df_location.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [37]:
df_18_25 = df1[df1['lpep_pickup_datetime'] == '2025-11-18']
df_18_25.shape

(0, 21)

In [38]:
df_18_25 = df1[df1['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()]
df_18_25.shape

(1773, 21)

In [39]:
location_revenue = (
    df_18_25
    .groupby('PULocationID')['total_amount']
    .sum()
)
location_revenue

PULocationID
7      343.60
10      87.24
14      53.83
15      39.50
17     110.25
        ...  
261     34.37
262     26.78
263     71.53
264     18.80
265     61.00
Name: total_amount, Length: 137, dtype: float64

In [40]:
location_revenue.idxmax(), location_revenue.max()

(np.int32(74), np.float64(9281.92))

In [42]:
# top_location = location_revenue.idxmax()

# df_18_25[df_18_25['PULocationID'] == top_location]

In [43]:
##Datframe type
location_revenue_df = (
    df_18_25
    .groupby('PULocationID')[['total_amount']]
    .sum()
)

In [44]:
df_loc_rev = location_revenue_df.merge(
    df_location,
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
)

In [47]:
df_loc_rev.loc[df_loc_rev['total_amount'].idxmax()]

total_amount              9281.92
LocationID                     74
Borough                 Manhattan
Zone            East Harlem North
service_zone            Boro Zone
Name: 39, dtype: object

In [56]:
df1_loc = df1.merge(df_location, left_on='PULocationID', 
                    right_on='LocationID', 
                    how='left')
df1_loc.shape

(46912, 25)

In [59]:
# november data:
df_nov_2025_EHN = df1_loc[
    (df1_loc['lpep_pickup_datetime'].dt.year == 2025) &
    (df1_loc['lpep_pickup_datetime'].dt.month == 11) &
    (df1_loc['Zone'] == 'East Harlem North')
    
]

df_nov_2025_EHN.shape

(12041, 25)

In [63]:
##Datframe type
drop_location_tip_df = (
    df_nov_2025_EHN
    .groupby('DOLocationID')[['tip_amount']]
    .sum()
)
drop_location_tip_df

,tip_amount
DOLocationID,
1,20.00
3,0.00
4,15.90
7,78.54
13,9.99
...,...
261,10.80
262,1488.96
263,2403.17


In [64]:
df_doloc_tip = drop_location_tip_df.merge(
    df_location,
    left_on='DOLocationID',
    right_on='LocationID',
    how='left'
)

In [65]:
df_doloc_tip.loc[df_doloc_tip['tip_amount'].idxmax()]

tip_amount                    4242.01
LocationID                        236
Borough                     Manhattan
Zone            Upper East Side North
service_zone              Yellow Zone
Name: 113, dtype: object

In [68]:
df_nov_2025_EHN.loc[df_nov_2025_EHN['tip_amount'].idxmax()]

VendorID                                   2
lpep_pickup_datetime     2025-11-30 16:30:27
lpep_dropoff_datetime    2025-11-30 16:41:29
store_and_fwd_flag                         N
RatecodeID                               1.0
PULocationID                              74
DOLocationID                             263
passenger_count                          1.0
trip_distance                           2.63
fare_amount                             14.2
extra                                    0.0
mta_tax                                  0.5
tip_amount                             81.89
tolls_amount                             0.0
ehail_fee                                NaN
improvement_surcharge                    1.0
total_amount                          100.34
payment_type                             1.0
trip_type                                1.0
congestion_surcharge                    2.75
cbd_congestion_fee                       0.0
LocationID                                74
Borough   

In [71]:
df_location[df_location['LocationID'] == 263]

,LocationID,Borough,Zone,service_zone
262,263,Manhattan,Yorkville West,Yellow Zone
